# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [4]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=True,
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [12]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=True,
    verbose=True
)

### Agent: Editor(편집자)

In [13]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=True,
    verbose=True
)

### Agent: Translator(번역자)

In [14]:
translator = Agent(
    role="번역자 (Translator)",
    goal="편집이 완료된 블로그 게시물을 요청한 언어로 정확하게 번역하여 사용자에게 제공합니다.",
    backstory="당신은 편집자(Editor)로부터 블로그 게시물을 받는 번역자입니다. "
              "당신은 다양한 언어에 능통한 숙련된 번역가이며, 특히 IT 도메인에 전문성이 있습니다. "
              "당신의 목표는 콘텐츠의 의미, 어조, 스타일을 유지하면서 텍스트를 정확하게 번역하는 것입니다. "
              "당신은 다양한 분야의 텍스트를 번역한 경험이 있으며, 각 언어의 문화적 뉘앙스를 이해하고 있습니다.",
    allow_delegation=True,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [15]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.\n"
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
)

### Task: Write(콘텐츠 작성)

In [16]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [17]:
edit = Task(
    description=(
        "1. 주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다.\n"
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

### Task: Translate(번역)

In [18]:
translate = Task(
    description=(
        "문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인이 끝나면 일본어로 번역합니다.\n"
    ),
    expected_output=(
        "일본어로 번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다."
    ),
    agent=translator,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [21]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, writer, editor, translator],
    tasks=[plan, translate, write, edit],
    process=Process.sequential,
    verbose=True
)

Overriding of current TracerProvider is not allowed


ValidationError: 1 validation error for Crew
  Attribute `manager_llm` or `manager_agent` is required when using hierarchical process. [type=missing_manager_llm_or_manager_agent, input_value={'agents': [Agent(role=...ical'>, 'verbose': True}, input_type=dict]

## Running the Crew

In [20]:
result = crew.kickoff(inputs={"topic": "인공지능"})

# Agent: Crew Manager
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.



# Agent: Crew Manager
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Create a comprehensive content plan document that includes an outline, audience analysis, SEO keywords, and resources.\", \"context\": \"The content should focus on the latest trends, key players, and noteworthy news in AI. Consider the target audience's interests and pain points. Include an introduction, key points, and a call to action. Also, incorporate SEO keywords and related data or sources.\", \"coworker\": \"\\ucf58\\ud150\\uce20 \\uae30\\ud68d\\uc790\"}"
## Tool Output: 

Error executing tool. coworker mentioned not found, 

- Display the results of your execution as markdown in the notebook.

In [22]:
result

CrewOutput(raw="```markdown\n# 人工知能の最先端世界を探る: トレンド、革新、そして主要プレーヤー\n\n## はじめに\n人工知能 (AI) の概要とその多様な産業への影響について述べます。最新のAIトレンドや開発状況を更新していくことの重要性についてもお話しします。\n\n## AIの現在のトレンド\n機械学習と深層学習の進展が顕著です。医療分野では、AIが診断、医薬品の発見、個別化医療において重要な役割を果たしています。また、オートメーション分野においても、ロボティクスや製造・プロセスオートメーションでのAIの応用が進んでいます。さらに、プライバシーやバイアス、倫理的意思決定の問題を考慮した倫理的AIに関する議論も進んでいます。\n\n## AI業界の主要プレーヤー\n主要なテクノロジー企業であるGoogle、IBM、MicrosoftがAIの分野で重要な貢献を続けています。それに加え、影響力のある新興スタートアップが続々と登場しており、AIの未来を形作る著名な研究者や思想リーダーがその先頭に立っています。\n\n## 注目すべきAIのニュースと発展\n最近のAI技術のブレークスルーや重要な研究論文、それに伴う発見が注目されています。また、AI開発に影響を及ぼす政府の政策や規制のアップデートもしっかりとフォローする必要があります。\n\n## AIにおける課題と挑戦\nAIによる自動化は仕事の置き換えに対する懸念を引き起こしています。また、AIの倫理性と透明性を確保すること、データのプライバシーやセキュリティに関する課題にも取り組む必要があります。\n\n## 結論と行動への呼びかけ\nAIのトレンドや主要プレーヤーを理解することの重要性を強調します。さらに深い研究を促すとともに、定期的なAIアップデートのためにニュースレターの購読をお勧めします。 \n\nこのコンテンツは、技術のプロフェッショナル、AIに興味を持つ方々、そしてAIが自分の業界に与える影響を理解しようとするビジネスリーダーを対象にしており、彼らの関心を引きつけつつ、SEOを通じて広範なリーチを目指しています。\n```\n\nThis blog post is now well-written, grammatically correct, and aligned

In [23]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# 人工知能の最先端世界を探る: トレンド、革新、そして主要プレーヤー

## はじめに
人工知能 (AI) の概要とその多様な産業への影響について述べます。最新のAIトレンドや開発状況を更新していくことの重要性についてもお話しします。

## AIの現在のトレンド
機械学習と深層学習の進展が顕著です。医療分野では、AIが診断、医薬品の発見、個別化医療において重要な役割を果たしています。また、オートメーション分野においても、ロボティクスや製造・プロセスオートメーションでのAIの応用が進んでいます。さらに、プライバシーやバイアス、倫理的意思決定の問題を考慮した倫理的AIに関する議論も進んでいます。

## AI業界の主要プレーヤー
主要なテクノロジー企業であるGoogle、IBM、MicrosoftがAIの分野で重要な貢献を続けています。それに加え、影響力のある新興スタートアップが続々と登場しており、AIの未来を形作る著名な研究者や思想リーダーがその先頭に立っています。

## 注目すべきAIのニュースと発展
最近のAI技術のブレークスルーや重要な研究論文、それに伴う発見が注目されています。また、AI開発に影響を及ぼす政府の政策や規制のアップデートもしっかりとフォローする必要があります。

## AIにおける課題と挑戦
AIによる自動化は仕事の置き換えに対する懸念を引き起こしています。また、AIの倫理性と透明性を確保すること、データのプライバシーやセキュリティに関する課題にも取り組む必要があります。

## 結論と行動への呼びかけ
AIのトレンドや主要プレーヤーを理解することの重要性を強調します。さらに深い研究を促すとともに、定期的なAIアップデートのためにニュースレターの購読をお勧めします。 

このコンテンツは、技術のプロフェッショナル、AIに興味を持つ方々、そしてAIが自分の業界に与える影響を理解しようとするビジネスリーダーを対象にしており、彼らの関心を引きつけつつ、SEOを通じて広範なリーチを目指しています。
```

This blog post is now well-written, grammatically correct, and aligned with the brand's voice, ready for publication.

In [42]:
result

CrewOutput(raw="The blog post has been refined for grammatical accuracy and alignment with the brand's voice as requested, presented in Markdown format. It provides a comprehensive overview of AI trends, news, industry players, and actionable insights.", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='1. 주어진 블로그 게시물(blog post)을 일본어로 번역합니다.\n', name=None, expected_output='번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다.', summary='1. 주어진 블로그 게시물(blog post)을 일본어로 번역합니다.\n...', raw='Please provide the content of the blog post that needs translation, so I can accurately translate it into Japanese for you.\n```', pydantic=None, json_dict=None, agent='번역자 (Translator)', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "2025년 3월 2일 서울 날씨"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 작가 (Content Writer)
## Task: 1. 콘텐츠 계획(content plan)을 활용하여 2025년 3월 2일 서울 날씨에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.
2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.
3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.
4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.
5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: What are the SEO keywords we need to integrate naturally into the blog post about 2025년 3월 2일 서울 날씨, and can you provide more details on the brand's voice for consistency?


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
To effectively plan for the blog post about the weather in Seoul on March 2, 2025, incorporating relevant SEO keywords is crucial for maximizing search engine visibility. Here are some suggested keywords to integrate naturally into the content:

1. **2025 Seoul Weather**
2. **March 2 Weather Forecast Seoul**


In [17]:
from IPython.display import Markdown
Markdown(result.raw)

The comprehensive content plan for the 2025년 3월 2일 서울 날씨 blog post includes a detailed outline (introduction, key points, conclusion, call to action), an audience analysis outlining interests and pain points, a list of SEO keywords, and suggested resources for information and inspiration.